In [ ]:
# Work 10
## Variables preparation - traffic transport infrastructure 

In [2]:
# loading libraries
import torch
torch.cuda.empty_cache() 
import os 
os.environ['MPLCONFIGDIR'] = os.getcwd() + "/configs/"
import matplotlib
from matplotlib import pyplot as plt
import numpy as np
import cv2
import urllib.request
from PIL import Image
import pandas as pd
import pyproj
from pathlib import Path
import joblib
import seaborn as sn
from scipy import stats
from tabulate import tabulate
import statsmodels.api as sm

In [3]:
# loading written functions
from mtp_function_yl import *

### Preparation of variables - traffic, transport, infrastructure

#### For accident locations

In [ ]:
# traffic count
# extracting time information from accident dataset
aa = pd.read_csv('/home/yelhe/script/mt/dataraw/dfaa.csv')
atime = aa[['Nr', 'Datum', 'Zeit']]
atime.loc[:, 'Date'] = pd.to_datetime(atime.loc[:, 'Datum'], format = "%Y-%m-%d")
atime.loc[:,'year'] = atime.loc[:,'Date'].dt.year
atime.loc[:,'month'] = atime.loc[:,'Date'].dt.month
atime.loc[:,'day'] = atime.loc[:,'Date'].dt.day
atime.loc[:,'week'] = atime.loc[:,'Date'].dt.isocalendar().week
atimegroup = atime.groupby(['year']).size().reset_index().rename(columns={0:'count'})
# atimegroup is 2019: 48; 2020: 74; 2021: 91; 2022: 137
#loading bicycle traffic count data 
btc19 = pd.read_csv('/home/yelhe/data/Traffic_network_variables/traffic_count/bicycle_traffic_count_2019.csv')
btc20 = pd.read_csv('/home/yelhe/data/Traffic_network_variables/traffic_count/bicycle_traffic_count_2020.csv')
btc21 = pd.read_csv('/home/yelhe/data/Traffic_network_variables/traffic_count/bicycle_traffic_count_2021.csv')
btc22 = pd.read_csv('/home/yelhe/script/mt/output/bic_tc_2022_new.csv')
btc22_0 = btc22.drop(['FK_STANDORT'], axis = 1)
btc22_0 = btc22_0.rename(columns = {'velo':'bicycle_traffic_count',
                                    'fk_zaehler':'FK_ZAEHLER'})
btc = pd.concat([btc19, btc20, btc21, btc22_0])
ac_btc = atime.merge(btc, how = 'left', on = ['year', 'month', 'week'])
# loading vehicle traffic count data
vtc19 = pd.read_csv('/home/yelhe/data/Traffic_network_variables/traffic_count/car_traffic_count_2019.csv')
vtc20 = pd.read_csv('/home/yelhe/data/Traffic_network_variables/traffic_count/car_traffic_count_2020.csv')
vtc21 = pd.read_csv('/home/yelhe/data/Traffic_network_variables/traffic_count/car_traffic_count_2021.csv')
vtc22 = pd.read_csv('/home/yelhe/script/mt/output/car_tc_2022.csv')
vtc22_0 = vtc22.rename(columns = {'AnzFahrzeuge':'car_traffic_count'})
vtc = pd.concat([vtc19, vtc20, vtc21, vtc22_0])
ac_vtc = atime.merge(vtc, how = 'left', on = ['year', 'month', 'week'])
# shortest distance data
disb = pd.read_csv('/home/yelhe/data/Traffic_network_variables/acc_bicyclecount.csv') 
disv = pd.read_csv('/home/yelhe/data/Traffic_network_variables/acc_carcount.csv')
disbu = disb[['Nr', 'fk_zaehler', 'distance']]
disbu = disbu.drop_duplicates()
disvu = disv[['Nr', 'MSID', 'distance']]
disvu = disvu.drop_duplicates()
disbu = disbu.rename(columns = {'fk_zaehler': 'FK_ZAEHLER'})
acc_bc = disbu.merge(ac_btc, how = 'left', on = ['Nr','FK_ZAEHLER'])
acc_vc = disvu.merge(ac_vtc, how = 'left', on = ['Nr', 'MSID'])
acc_bc.to_csv('/home/yelhe/data/Traffic_network_variables/traffic_count/bicyclecount_new.csv')
acc_vc.to_csv('/home/yelhe/data/Traffic_network_variables/traffic_count/carcount_new.csv')

In [ ]:
# (need to check hard disk)
# bicycle traffic count
wrbc = pd.read_csv('/home/yelhe/data/Traffic_network_variables/traffic_count/bicyclecount_u.csv')
wrbci = wrbc.set_index('Unnamed: 0')
wrbcf = acc_bc.join(wrbc,  lsuffix='', rsuffix='r',)
wrbcf = wrbcf[['Nr', 'bicycle_traffic_countr', 'distancer']]
wrbcf = wrbcf.dropna()
wrbcf = wrbcf.rename(columns = {'bicycle_traffic_countr':'bicycle_traffic_count',
                               'distancer':'distance'})
wrbcf.to_csv('/home/yelhe/data/Traffic_network_variables/traffic_count/bc_nu.csv')
# vehicle traffic count
wrvc = pd.read_csv('/home/yelhe/data/Traffic_network_variables/traffic_count/carcount_u.csv')
wrvci = wrvc.set_index('Unnamed: 0')
wrvcf = acc_vc.join(wrvc,  lsuffix='', rsuffix='r')
wrvcf = wrvcf[['Nr', 'car_traffic_countr', 'distancer']]
wrvcf = wrvcf.dropna()
wrvcf = wrvcf.rename(columns = {'car_traffic_countr':'car_traffic_count',
                               'distancer':'distance'})
wrvcf.to_csv('/home/yelhe/data/Traffic_network_variables/traffic_count/vc_nu.csv')

In [ ]:
# bicycle data for 2022
bi22 = bi2022.drop(['FUSS_IN', 'FUSS_OUT', 'OST', 'NORD'], axis = 1)
bi22 = bi22.fillna(0)
bi22['velo'] = (bi22['VELO_IN'] + bi22['VELO_OUT']) * 1.185
biw = bi22.copy(deep = True)
biw.loc[:, 'Date'] = pd.to_datetime(biw.loc[:, 'DATUM'], 
                                              format = "%Y-%m-%dT%H:%M",
                                              errors='coerce')
biw.loc[:,'year'] = biw.loc[:,'Date'].dt.year
biw.loc[:,'month'] = biw.loc[:,'Date'].dt.month
biw.loc[:,'day'] = biw.loc[:,'Date'].dt.day
biw.loc[:, 'week'] = biw.loc[:, 'Date'].dt.isocalendar().week

biwsm = biw.groupby(['year', 'month', 'week' ])['velo'].sum().reset_index()
# biwsm.to_csv('/home/yelhe/script/mt/output/bic_tc_2022_weekmean.csv', index = False)
biws = biw.groupby(['FK_STANDORT', 'year', 'month', 'week' ])['velo'].sum().reset_index()
# bicycle counting station locations
bcl = pd.read_csv('/home/yelhe/data/Traffic_network_variables/countstation_bicycle.csv')
bcln = bcl[['fk_zaehler','id1']]
bcln = bcln.rename(columns = {'id1': 'FK_STANDORT'})
biws_id = pd.merge(biws, bcln, how = 'left', on = ['FK_STANDORT'])
biwc_v = biws.loc[biws['velo'] != 0]
biwc_mean = biwc_v.groupby(['week' ])['velo'].mean().reset_index()
# vehicle data for 2022
ca2022 = pd.read_csv('/home/yelhe/data/Traffic_network_variables/sid_dav_verkehrszaehlung_miv_od2031_2022.csv')
bi2022 = pd.read_csv('/home/yelhe/data/Traffic_network_variables/2022_verkehrszaehlungen_werte_fussgaenger_velo.csv')
caw = ca2022.copy(deep = True)
caw.loc[:, 'Date'] = pd.to_datetime(caw.loc[:, 'MessungDatZeit'], 
                                              format = "%Y-%m-%dT%H:%M:%S",
                                              errors='coerce')
caw.loc[:,'year'] = caw.loc[:,'Date'].dt.year
caw.loc[:,'month'] = caw.loc[:,'Date'].dt.month
caw.loc[:,'day'] = caw.loc[:,'Date'].dt.day
caw.loc[:, 'week'] = caw.loc[:, 'Date'].dt.isocalendar().week
caws = caw.groupby(['MSID', 'year', 'month', 'day', 'week' ])['AnzFahrzeuge'].sum().reset_index()
cawsm = caws.groupby(['MSID', 'year', 'month', 'week' ])['AnzFahrzeuge'].sum().reset_index()
cawsmm = caws.groupby(['year', 'month', 'week' ])['AnzFahrzeuge'].mean().reset_index()

In [ ]:
# other variables
aa = pd.read_csv('/home/yelhe/script/mt/dataraw/dfaa.csv')
# loading traffic transport infrastructure variables
# data of distance to features
busl = pd.read_csv('/home/yelhe/data/Traffic_network_variables/h/acc_busline.csv')
traml = pd.read_csv('/home/yelhe/data/Traffic_network_variables/h/acc_tramrail.csv')
trainl = pd.read_csv('/home/yelhe/data/Traffic_network_variables/h/acc_sbahn_h.csv')
plight = pd.read_csv('/home/yelhe/data/Traffic_network_variables/h/acc_publiclight.csv')
station = pd.read_csv('/home/yelhe/data/Traffic_network_variables/h/acc_station_h.csv')
parkcar = pd.read_csv('/home/yelhe/data/Traffic_network_variables/h/acc_parking_car_space.csv')
parktw = pd.read_csv('/home/yelhe/data/Traffic_network_variables/h/acc_parking_twowheeler.csv')
pedstrainc = pd.read_csv('/home/yelhe/data/Traffic_network_variables/acc_pedestraincrossing_n.csv') # Greg
stop = pd.read_csv('/home/yelhe/data/Traffic_network_variables/acc_stopsign_n.csv') # Greg
# data of road width and road surface information
roadinfo = pd.read_csv('/home/yelhe/data/Traffic_network_variables/acc_roadsurfacewidth.csv') # Greg
# data of speedlimit
speedlimit = pd.read_csv('/home/yelhe/data/Traffic_network_variables/acc_speedlimit.csv') # Greg
# data of traffic volume/exposure
gvm = pd.read_csv('/home/yelhe/data/Traffic_network_variables/h/acc_gvm.csv')
# data of urban statistics zone
trafficarea = pd.read_csv('/home/yelhe/data/Traffic_network_variables/h/acc_trafficarea_h.csv')
statzone = pd.read_csv('/home/yelhe/data/Traffic_network_variables/h/acc_statzonen_h.csv')
# data of curb plan
curb = pd.read_csv('/home/yelhe/data/Traffic_network_variables/h/acc_curb.csv')
# previously generated data of traffic count
bc = pd.read_excel('/home/yelhe/data/Traffic_network_variables/traffic_count/bicyclecount_1.xlsx')
vc = pd.read_excel('/home/yelhe/data/Traffic_network_variables/traffic_count/carcount_1.xlsx')

In [ ]:
# merging all these variables into one file
ttv = pd.DataFrame()
# bus line
buslg = busl.groupby(['Nr', 'distance']).size().reset_index().rename(columns={0:'count'})
buslg = buslg.rename(columns = {'distance': 'dbusl'})
buslg = buslg.drop(['count'], axis = 1)
ttv['Nr'] = aa['Nr']
ttv = ttv.merge(buslg, on = ['Nr'])
# tram line
tramlg = traml.groupby(['Nr', 'distance']).size().reset_index().rename(columns={0:'count'})
tramlg = tramlg.rename(columns = {'distance': 'dtraml'})
tramlg = tramlg.drop(['count'], axis = 1)
ttv = ttv.merge(tramlg, on = ['Nr'])
# railway
trainlg = trainl.groupby(['Nr', 'distance']).size().reset_index().rename(columns={0:'count'})
trainlg = trainlg.rename(columns = {'distance': 'dtrainl'})
trainlg = trainlg.drop(['count'], axis = 1)
ttv = ttv.merge(trainlg, on = ['Nr'])
# public light
plightg = plight.groupby(['Nr', 'distance']).size().reset_index().rename(columns={0:'count'})
plightg = plightg.rename(columns = {'distance': 'dplight'})
plightg = plightg.drop(['count'], axis = 1)
ttv = ttv.merge(plightg, on = ['Nr'])
# public transport station
stationg = station.groupby(['Nr', 'distance']).size().reset_index().rename(columns={0:'count'})
stationg = stationg.rename(columns = {'distance': 'dstation'})
stationg = stationg.drop(['count'], axis = 1)
ttv = ttv.merge(stationg, on = ['Nr'])
# car parking space
parkcarg = parkcar.groupby(['Nr', 'distance']).size().reset_index().rename(columns={0:'count'})
parkcarg = parkcarg.rename(columns = {'distance': 'dparkcar'})
parkcarg = parkcarg.drop(['count'], axis = 1)
ttv = ttv.merge(parkcarg, on = ['Nr'])
# two-wheeler parking space
parktwg = parktw.groupby(['Nr', 'distance']).size().reset_index().rename(columns={0:'count'})
parktwg = parktwg.rename(columns = {'distance': 'dparktw'})
parktwg = parktwg.drop(['count'], axis = 1)
ttv = ttv.merge(parktwg, on = ['Nr'])
# pedestrian crossing
pedcrg = pedstrainc.groupby(['Nr', 'distance']).size().reset_index().rename(columns={0:'count'})
pedcrg = pedcrg.rename(columns = {'distance': 'dpedcro'})
pedcrg = pedcrg.drop(['count'], axis = 1)
ttv = ttv.merge(pedcrg, on = ['Nr'])
# stop sign
stopsg = stop.groupby(['Nr', 'distance']).size().reset_index().rename(columns={0:'count'})
stopsg = stopsg.rename(columns = {'distance': 'dstopsign'})
stopsg = stopsg.drop(['count'], axis = 1)
ttv = ttv.merge(stopsg, on = ['Nr'])
# curb (plan)
curbg = curb.groupby(['Nr', 'distance']).size().reset_index().rename(columns={0:'count'})
curbg = curbg.rename(columns = {'distance': 'dcurb'})
curbg = curbg.drop(['count'], axis = 1)
ttv = ttv.merge(curbg, on = ['Nr'])
# curb type (plan)
curbty = curb[['Nr', 'distance', 'abschluss']]
curbty = curbty.rename(columns = {'abschluss':'curbtype', 'distance':'dcurb'})
curbty = curbty.drop_duplicates()
ttv = ttv.merge(curbty, on = ['Nr', 'dcurb'])
# traffic
gvmg = gvm.groupby(['Nr', 'distance']).size().reset_index().rename(columns={0:'count'})
gvmg = gvmg.rename(columns = {'distance': 'dgvm'})
gvmg = gvmg.drop(['count'], axis = 1)
ttv = ttv.merge(gvmg, on = ['Nr'])
gvmvalue = gvm[['Nr', 'DWV_ANZ_Q', 'MSP_ANZ_Q', 'ASP_ANZ_Q', 'distance']]
gvmvalue = gvmvalue.rename(columns = {'DWV_ANZ_Q':'gvm_dwv',
                                      'MSP_ANZ_Q':'gvm_msp',
                                      'ASP_ANZ_Q':'gvm_asp',
                                      'distance':'dgvm'})
ttv = ttv.merge(gvmvalue, on = ['Nr', 'dgvm'])
ttv = ttv.drop_duplicates()
ttv = ttv.reset_index()
ttv = ttv.drop(['index'], axis = 1)
# statistical zone
statzoneg = statzone[['Nr', 'stznr', 'qnr', 'knr']]
statzoneg = statzoneg.rename(columns = {'stznr':'z_stznr', 'qnr':'z_qnr', 'knr':'z_knr'})
ttv = ttv.merge(statzoneg, on = ['Nr'])
# traffic area
trafficarea.groupby(['zonentyp', 'zonentyp_t']).size()
tareag = trafficarea[['Nr', 'zonentyp_t', 'distance']]
tareag = tareag.rename(columns = {'zonentyp_t':'trafficarea',
                                  'distance':'dtrafficarea'})
ttv = ttv.merge(tareag, on = ['Nr'])
# road information
rig = roadinfo[['Nr', 'road_width', 'surface', 'distance']]
rig = rig.rename(columns = {'road_width':'r_width',
                               'distance':'droad',
                               'surface':'r_surface'})
ttv = ttv.merge(rig, on = ['Nr'])
# speed limit
slg = speedlimit[['Nr', 'speed', 'distance']]
slg = slg.rename(columns = {'speed':'speedlimit',
                            'distance':'droad_sl'})
ttv = ttv.merge(slg, on = ['Nr'])
# bicycle traffic count
bc = bc.drop(['Unnamed: 0'], axis = 1)
bcg = bc.groupby(['Nr', 'distance']).size().reset_index().rename(columns={0:'count'})
bcg = bcg.rename(columns = {'distance': 'd_bicyclecount'})
bcg = bcg.drop(['count'], axis = 1)
ttv = ttv.merge(bcg, on = ['Nr'], how = 'left')
bcgv = bc[['Nr', 'bicycle_traffic_count', 'distance']]
bcgv = bcgv.rename(columns = {'bicycle_traffic_count':'bicyclecount',
                              'distance':'d_bicyclecount'})
ttv = ttv.merge(bcgv, on = ['Nr', 'd_bicyclecount'], how = 'left')
# vehicle traffic count
vcg = vc.groupby(['Nr', 'distance']).size().reset_index().rename(columns={0:'count'})
vcg = vcg.rename(columns = {'distance': 'd_carcount'})
vcg = vcg.drop(['count'], axis = 1)
vcg = vcg.drop_duplicates()
ttv = ttv.merge(vcg, on = ['Nr'], how = 'left')
vcgv = vc[['Nr', 'car_traffic_count', 'distance']]
vcgv = vcgv.rename(columns = {'car_traffic_count':'carcount',
                              'distance':'d_carcount'})
ttv = ttv.merge(vcgv, on = ['Nr', 'd_carcount'], how = 'left')
# bicycle and pedestrian route network
vfpath = pd.read_csv('/home/yelhe/data/Traffic_network_variables/h/acc_velofusspath.csv')
vfpg = vfpath.groupby(['Nr', 'distance']).size().reset_index().rename(columns={0:'count'})
vfpg = vfpg.rename(columns = {'distance': 'dvfpath'})
vfpg = vfpg.drop(['count'], axis = 1)
ttv = ttv.merge(vfpg, on = ['Nr'])
# saving all traffic transport infrastructure variables into one single file
ttv.to_csv('/home/yelhe/data/Traffic_network_variables/ttvariables_sum_final.csv', index = False)

#### For random pseudo locations

In [ ]:
# traffic count
# bicycle traffic dount 
btc19 = pd.read_csv('/home/yelhe/data/Traffic_network_variables/traffic_count/bicycle_traffic_count_2019.csv')
btc20 = pd.read_csv('/home/yelhe/data/Traffic_network_variables/traffic_count/bicycle_traffic_count_2020.csv')
btc21 = pd.read_csv('/home/yelhe/data/Traffic_network_variables/traffic_count/bicycle_traffic_count_2021.csv')
btc22 = pd.read_csv('/home/yelhe/script/mt/output/bic_tc_2022_new.csv')
btc22_0 = btc22.drop(['FK_STANDORT'], axis = 1)
btc22_0 = btc22_0.rename(columns = {'velo':'bicycle_traffic_count',
                                    'fk_zaehler':'FK_ZAEHLER'})
btc = pd.concat([btc19, btc20, btc21, btc22_0])
# vehicle traffic count
vtc19 = pd.read_csv('/home/yelhe/data/Traffic_network_variables/traffic_count/car_traffic_count_2019.csv')
vtc20 = pd.read_csv('/home/yelhe/data/Traffic_network_variables/traffic_count/car_traffic_count_2020.csv')
vtc21 = pd.read_csv('/home/yelhe/data/Traffic_network_variables/traffic_count/car_traffic_count_2021.csv')
vtc22 = pd.read_csv('/home/yelhe/script/mt/output/car_tc_2022.csv')
vtc22_0 = vtc22.rename(columns = {'AnzFahrzeuge':'car_traffic_count'})
vtc = pd.concat([vtc19, vtc20, vtc21, vtc22_0])
# shortest distance 
disb = pd.read_csv('/home/yelhe/data/Traffic_network_variables/rp_bicyclecount.csv') 
disv = pd.read_csv('/home/yelhe/data/Traffic_network_variables/rp_carcount.csv') 
raw = pd.read_csv('/home/yelhe/data/Traffic_network_variables/random_pseudo_new.csv')
raw = raw.reset_index()
raw = raw[['rand_point', 'index']]
raw = raw.rename(columns = {'index':'id'})
raw.to_csv('/home/yelhe/data/Traffic_network_variables/idconversion.csv', index = False)
rp_koord = pd.read_csv('/home/yelhe/script/mt/output/rp_koord.csv')
# ranp = pd.read_csv('/home/yelhe/data/Traffic_network_variables/ran_pseudo_new_withgeo.csv')
# ranp = ranp[['rand_point', 'x', 'y']]
# ranp = ranp.rename(columns = {'x':'xcoord','y':'ycoord'})
# ranp = ranp.assign(lat = None, lon = None)
# transformer = pyproj.Transformer.from_crs("EPSG:2056", "EPSG:4326")
# a, b = transformer.transform(ranp[['xcoord']], ranp[['ycoord']])
# ranp = ranp.assign(lon = b, lat = a)
# df1 = pd.merge(raw, ranp, on = ['rand_point'])
# df2 = pd.merge(df1, rp_koord, on = ['id'])
#bicycle traffic count
disb = pd.read_csv('/home/yelhe/data/Traffic_network_variables/rp_bicyclecount.csv') 
disb = pd.merge(raw, disb, how = 'left', on = ['rand_point'])
disbu = disb[['id', 'fk_zaehler', 'distance']]
disbu = disbu.drop_duplicates()
disbu = disbu.rename(columns = {'fk_zaehler': 'FK_ZAEHLER'})
btcn = btc[['FK_ZAEHLER','bicycle_traffic_count']]
btcn = btcn.dropna()
btcn = btcn.loc[btcn['bicycle_traffic_count'] != 0]
btcm = btcn.groupby(['FK_ZAEHLER'])['bicycle_traffic_count'].mean().reset_index()
rp_bc = btcm.merge(disbu, how = 'left', on = ['FK_ZAEHLER'])
rp_bcr = rp_bc.dropna(subset = ['id'])
rp_bcr.to_csv('/home/yelhe/data/Traffic_network_variables/traffic_count/bicyclecount_rp.csv')
rp_bcrm = rp_bcr.groupby(['id', 'distance'])['bicycle_traffic_count'].mean().reset_index()
rp_bcrm.to_csv('/home/yelhe/data/Traffic_network_variables/traffic_count/bicyclecountm_rp.csv')
# vehicle traffic count
disv = pd.read_csv('/home/yelhe/data/Traffic_network_variables/rp_carcount.csv')
disv = pd.merge(raw, disv, how = 'left', on = ['rand_point'])
disvu = disv[['id', 'MSID', 'distance']]
disvu = disvu.drop_duplicates()
vtcn = vtc[['MSID','car_traffic_count']]
vtcn = vtcn.dropna()
vtcn = vtcn.loc[vtcn['car_traffic_count'] != 0]
vtcm = vtcn.groupby(['MSID'])['car_traffic_count'].mean().reset_index()
rp_vc = vtcm.merge(disvu, how = 'left', on = ['MSID'])
rp_vcr = rp_vc.dropna(subset = ['id'])
rp_vcr.to_csv('/home/yelhe/data/Traffic_network_variables/traffic_count/carcount_rp.csv')
rp_vcrm = rp_vcr.groupby(['id', 'distance'])['car_traffic_count'].mean().reset_index()
rp_vcrm.to_csv('/home/yelhe/data/Traffic_network_variables/traffic_count/carcountm_rp.csv')

In [ ]:
# other variables
# distance to features
busl = pd.read_csv('/home/yelhe/data/Traffic_network_variables/rp_busline.csv')
traml = pd.read_csv('/home/yelhe/data/Traffic_network_variables/rp_tramrail.csv')
trainl = pd.read_csv('/home/yelhe/data/Traffic_network_variables/rp_sbahn.csv')
plight = pd.read_csv('/home/yelhe/data/Traffic_network_variables/rp_publiclight.csv')
station = pd.read_csv('/home/yelhe/data/Traffic_network_variables/rp_station.csv')
parkcar = pd.read_csv('/home/yelhe/data/Traffic_network_variables/rp_parking_car_space.csv')
parktw = pd.read_csv('/home/yelhe/data/Traffic_network_variables/rp_parking_twowheeler.csv')
pedstrainc = pd.read_csv('/home/yelhe/data/Traffic_network_variables/rp_pedestriancrossing.csv') # Greg
stop = pd.read_csv('/home/yelhe/data/Traffic_network_variables/rp_stopsign.csv') # Greg
# road width and road surface information
roadinfo = pd.read_csv('/home/yelhe/data/Traffic_network_variables/rp_roadsurfacewidth.csv') # Greg
# speedlimit
speedlimit = pd.read_csv('/home/yelhe/data/Traffic_network_variables/rp_speedlimit.csv') # Greg
# traffic volume/exposure
gvm = pd.read_csv('/home/yelhe/data/Traffic_network_variables/rp_gvm.csv')
# area statistics zone
trafficarea = pd.read_csv('/home/yelhe/data/Traffic_network_variables/rp_traffic_area.csv')
statzone = pd.read_csv('/home/yelhe/data/Traffic_network_variables/rp_statzonen.csv')
# curb info (plan)
curb = pd.read_csv('/home/yelhe/data/Traffic_network_variables/rp_curb.csv')
# identifier correspondence
idc = pd.read_csv('/home/yelhe/data/Traffic_network_variables/idconversion.csv')
# traffic count
bc = pd.read_excel('/home/yelhe/data/Traffic_network_variables/traffic_count/bicyclecountm_rp.xlsx')
vc = pd.read_excel('/home/yelhe/data/Traffic_network_variables/traffic_count/carcountm_rp.xlsx')

In [ ]:
# merging all these variables into one file
ttv = pd.DataFrame()
# bus line
buslg = busl.groupby(['rand_point', 'distance']).size().reset_index().rename(columns={0:'count'})
buslg = buslg.rename(columns = {'distance': 'dbusl'})
buslg = buslg.drop(['count'], axis = 1)
ttv['rand_point'] = idc['rand_point']
ttv = ttv.merge(buslg, on = ['rand_point'])
# tram rail
tramlg = traml.groupby(['rand_point', 'distance']).size().reset_index().rename(columns={0:'count'})
tramlg = tramlg.rename(columns = {'distance': 'dtraml'})
tramlg = tramlg.drop(['count'], axis = 1)
ttv = ttv.merge(tramlg, on = ['rand_point'])
# train rail
trainlg = trainl.groupby(['rand_point', 'distance']).size().reset_index().rename(columns={0:'count'})
trainlg = trainlg.rename(columns = {'distance': 'dtrainl'})
trainlg = trainlg.drop(['count'], axis = 1)
ttv = ttv.merge(trainlg, on = ['rand_point'])
# public light
plightg = plight.groupby(['rand_point', 'distance']).size().reset_index().rename(columns={0:'count'})
plightg = plightg.rename(columns = {'distance': 'dplight'})
plightg = plightg.drop(['count'], axis = 1)
ttv = ttv.merge(plightg, on = ['rand_point'])
# public transport station
stationg = station.groupby(['rand_point', 'distance']).size().reset_index().rename(columns={0:'count'})
stationg = stationg.rename(columns = {'distance': 'dstation'})
stationg = stationg.drop(['count'], axis = 1)
ttv = ttv.merge(stationg, on = ['rand_point'])
# car parking space
parkcarg = parkcar.groupby(['rand_point', 'distance']).size().reset_index().rename(columns={0:'count'})
parkcarg = parkcarg.rename(columns = {'distance': 'dparkcar'})
parkcarg = parkcarg.drop(['count'], axis = 1)
ttv = ttv.merge(parkcarg, on = ['rand_point'])
# two-wheeler parking space
parktwg = parktw.groupby(['rand_point', 'distance']).size().reset_index().rename(columns={0:'count'})
parktwg = parktwg.rename(columns = {'distance': 'dparktw'})
parktwg = parktwg.drop(['count'], axis = 1)
ttv = ttv.merge(parktwg, on = ['rand_point'])
# pedestrian crossing
pedcrg = pedstrainc.groupby(['rand_point', 'distance']).size().reset_index().rename(columns={0:'count'})
pedcrg = pedcrg.rename(columns = {'distance': 'dpedcro'})
pedcrg = pedcrg.drop(['count'], axis = 1)
ttv = ttv.merge(pedcrg, on = ['rand_point'])
# stop sign
stopsg = stop.groupby(['rand_point', 'distance']).size().reset_index().rename(columns={0:'count'})
stopsg = stopsg.rename(columns = {'distance': 'dstopsign'})
stopsg = stopsg.drop(['count'], axis = 1)
ttv = ttv.merge(stopsg, on = ['rand_point'])
# curb (plan)
curbg = curb.groupby(['rand_point', 'distance']).size().reset_index().rename(columns={0:'count'})
curbg = curbg.rename(columns = {'distance': 'dcurb'})
curbg = curbg.drop(['count'], axis = 1)
ttv = ttv.merge(curbg, on = ['rand_point'])
# curb type (plan)
curbty = curb[['rand_point', 'distance', 'abschluss']]
curbty = curbty.rename(columns = {'abschluss':'curbtype', 'distance':'dcurb'})
curbty = curbty.drop_duplicates()
ttv = ttv.merge(curbty, on = ['rand_point', 'dcurb'])
# traffic 
gvmg = gvm.groupby(['rand_point', 'distance']).size().reset_index().rename(columns={0:'count'})
gvmg = gvmg.rename(columns = {'distance': 'dgvm'})
gvmg = gvmg.drop(['count'], axis = 1)
ttv = ttv.merge(gvmg, on = ['rand_point'])
gvmvalue = gvm[['rand_point', 'DWV_ANZ_Q', 'MSP_ANZ_Q', 'ASP_ANZ_Q', 'distance']]
gvmvalue = gvmvalue.rename(columns = {'DWV_ANZ_Q':'gvm_dwv',
                                      'MSP_ANZ_Q':'gvm_msp',
                                      'ASP_ANZ_Q':'gvm_asp',
                                      'distance':'dgvm'})
ttv = ttv.merge(gvmvalue, on = ['rand_point', 'dgvm'])
ttv = ttv.drop_duplicates()
ttv = ttv.reset_index()
ttv = ttv.drop(['index'], axis = 1)
# statistical zone
statzoneg = statzone[['rand_point', 'stznr', 'qnr', 'knr']]
statzoneg = statzoneg.rename(columns = {'stznr':'z_stznr', 'qnr':'z_qnr', 'knr':'z_knr'})
ttv = ttv.merge(statzoneg, on = ['rand_point'])
# trafficc area
trafficarea.groupby(['zonentyp', 'zonentyp_t']).size()
tareag = trafficarea[['rand_point', 'zonentyp_t', 'distance']]
tareag = tareag.rename(columns = {'zonentyp_t':'trafficarea',
                                  'distance':'dtrafficarea'})
ttv = ttv.merge(tareag, on = ['rand_point'])
# road information
rig = roadinfo[['rand_point', 'road_width', 'surface', 'distance']]
rig = rig.rename(columns = {'road_width':'r_width',
                               'distance':'droad',
                               'surface':'r_surface'})
ttv = ttv.merge(rig, on = ['rand_point'])
# speed limit
slg = speedlimit[['rand_point', 'speed', 'distance']]
slg = slg.rename(columns = {'speed':'speedlimit',
                            'distance':'droad_sl'})
ttv = ttv.merge(slg, on = ['rand_point'])
ttvrp = pd.merge(idc, ttv, on = ['rand_point'], how = 'left')
# ttvrp.to_csv('/home/yelhe/data/Traffic_network_variables/ttvrp.csv', index = False)
# ttvrp = pd.read_excel('/home/yelhe/data/Traffic_network_variables/ttvrp.xlsx')
# bicycle traffic count
bcg = bc.rename(columns = {'distance': 'd_bicyclecount',
                           'bicycle_traffic_count':'bicyclecount'})
ttv_0 = ttvrp.merge(bcg, on = ['id'], how = 'left')
# vehicle traffic count
vcg = vc.rename(columns = {'distance': 'd_carcount',
                           'car_traffic_count':'carcount'})
ttv_1 = ttv_0.merge(vcg, on = ['id'], how = 'left')
# ttv = pd.read_csv('/home/yelhe/data/Traffic_network_variables/ttvariables_sum_withtrafficcount.csv')
# bicycle and pedestrian route network
vfpath = pd.read_csv('/home/yelhe/data/Traffic_network_variables/rp_veloandfusspath.csv')
vfpath1 = pd.merge(idc, vfpath, on = ['rand_point'], how = 'left')
vfpg = vfpath1.groupby(['id', 'distance']).size().reset_index().rename(columns={0:'count'})
vfpg = vfpg.rename(columns = {'distance': 'dvfpath'})
vfpg = vfpg.drop(['count'], axis = 1)
ttvall = ttv_1.merge(vfpg, on = ['id'])
ttvallfinal = ttvall.drop(['rand_point'],axis = 1)
# saving all traffic transport infrastructure variables into one single file
ttvallfinal.to_csv('/home/yelhe/data/Traffic_network_variables/ttvariables_rp_final.csv', index = False)